<a href="https://colab.research.google.com/github/Makoto-Noda/google-Colab/blob/main/CatsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/jeffd23/catdognet-keras-convnet-starter

In [ ]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [8]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 98% 800M/814M [00:13<00:00, 136MB/s]
100% 814M/814M [00:13<00:00, 64.4MB/s]


トレーニングデータ・テストデータの作成

In [13]:
TRAIN_DIR = './train/'
TEST_DIR = './test/'

In [14]:
ROWS = 64
COLS = 64
CHANNELS = 3

In [16]:
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [17]:
train_images = train_dogs[:1000] + train_cats[:1000]
random.shuffle(train_images)
test_images =  test_images[:25]